<a href="https://colab.research.google.com/github/rbdus0715/Machine-Learning/blob/main/study/cuda/01.intro-cuda/10.summation_with_error_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **(1) 에러 핸들링**

- 컴파일 타임 에러 : 문법이 틀려 발생한 에러
- 런타임 에러 : 프로그램 실행 중 발생한 에러

### **(2) 쿠다 에러 핸들링**
- 커널 런치를 제외한 거의 모든 CUDA 함수는 cudaError.cudaError를 반환함
```cpp
cudaError cuda_function(....)
```
- cudaError는 enum형
    - cudaSuccess : 성공
    - 실패했을 경우 : cudaError를 cudaGetErrorString 함수로 전달
- 이러한 런타임 핸들링이 쿠다 프로그래밍에 있어서 굉장히 중요함
    - 모든 함수마다 에러 체킹을 필수적으로 하기

- 에러 체킹 방법 1
```cpp
error = cudaMalloc((int **)&d_a, NO_BYTES);
if(error != cudaSuccess) {
    fprintf(stderr, "%s \n", cudaGetErrorString(error));
}
```
- 에러 체킹 방법 2
```cpp
// 에러 체크 함수
#define gpuErrchk(ans) {gpuAssert((ans), __FILE__, __LINE);}
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort = true)
{
    if(code != cudaSuccess)
    {
        fprintf(stderr, "GPUassert: %s %s %d \n", cudaGetErrorString(code), file, line);
        if(abort) exit(code);
    }
}
// 사용 예시
gpuErrchk(cudaMalloc((int**)&d_a, NO_BYTE);
```


In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

In [3]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>

// for random initialize
#include <stdlib.h>
#include <time.h>

// for memset
#include <cstring>

__global__ void sum_array_gpu(int * a, int * b, int * c, int size)
{
    int gid = blockIdx.x * blockDim.x + threadIdx.x;

    if(gid < size)
    {
        c[gid] = a[gid] + b[gid];
    }
}

void sum_array_cpu(int * a, int * b, int * c, int size)
{
    for(int i=0; i<size; i++)
    {
        c[i] = a[i] + b[i];
    }
}

void compare_arrays(int * a, int * b, int size)
{
    for (int i=0; i<size; i++)
    {
        if(a[i] != b[i])
        {
            printf("Array are different \n");
            return;
        }
    }
    printf("Arrays are same \n");
}

#define gpuErrchk(ans) {gpuAssert((ans), __FILE__, __LINE__);}
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort = true)
{
    if(code != cudaSuccess)
    {
        fprintf(stderr, "GPUassert: %s %s %d \n", cudaGetErrorString(code), file, line);
        if(abort) exit(code);
    }
}

int main() {
    int size = 1 << 25;
    int block_size = 1024;
    cudaError error;

    size_t NO_BYTES = size * sizeof(int);

    // host pointers
    // cpu_results cpu 계산 결과를 저장
    int * h_a, * h_b, * gpu_results, * cpu_results;

    // host에 메모리 할당
    h_a = (int*)malloc(NO_BYTES);
    h_b = (int*)malloc(NO_BYTES);
    gpu_results = (int*)malloc(NO_BYTES);
    cpu_results = (int*)malloc(NO_BYTES);

    // host 포인터 initialize
    time_t t;
    srand((unsigned)time(&t));
    for(int i=0; i<size; i++) {
        h_a[i] = (int)(rand() & 0xFF);
    }
    for(int i=0; i<size; i++) {
        h_b[i] = (int)(rand() & 0xFF);
    }

    // host calculation
    sum_array_cpu(h_a, h_b, cpu_results, size);

    memset(gpu_results, 0, NO_BYTES);

    // device pointer
    int * d_a, * d_b, * d_c;

    /////////////////////////////////////////////////////
    // error check (1)
    // 쿠다 함수 적용할 때마다 error를 반환받는다
    gpuErrchk(cudaMalloc((int **)&d_a, NO_BYTES));
    gpuErrchk(cudaMalloc((int **)&d_b, NO_BYTES));
    gpuErrchk(cudaMalloc((int **)&d_c, NO_BYTES));

    // transfer
    cudaMemcpy(d_a, h_a, NO_BYTES, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, NO_BYTES, cudaMemcpyHostToDevice);

    // 커널 크기
    dim3 block(block_size);
    // size가 완벽하게 block.x로 나눠지지 않을 때는 보통 1 grid size를 추가해준다.
    dim3 grid((size/block.x) + 1);

    /////////////////////////////////////////////////////
    /// 커널 런치는 아무것도 반환하지 않는다 (cudaError도 반환하지 않는다)
    sum_array_gpu<<<grid, block>>>(d_a, d_b, d_c, size);
    cudaDeviceSynchronize();

    cudaMemcpy(gpu_results, d_c, NO_BYTES, cudaMemcpyDeviceToHost);

    // array comparison
    compare_arrays(gpu_results, cpu_results, size);

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    free(h_a);
    free(h_b);
    free(gpu_results);

    cudaDeviceReset();
    return 0;
}

Arrays are same 

